In [1]:
using Plots
using Statistics
using Surrogates

include("../../scan_hyperparameters_emi4D_functions.jl")

main (generic function with 1 method)

In [2]:
@load "../../models/2023-02-02_16-06-47_w=100_d=5_activation=sigmoid_bs=1024_lr=0.001_dr=0.0_1.jld2" m

1-element Vector{Symbol}:
 :m

```
DVAR1: Bmax  [ 8.5, 12.0 ]
DVAR2: mX1  [ 5, 140 ]
DVAR3: mX2  [ 15, 160 ]
DVAR4: mY1  [ 1.005, 1.7 ]
DVAR5: mY2  [ 1.055, 1.85 ]
DVAR6: mtau1  [ 1, 500 ]
DVAR7: mtau2  [ 1, 500 ]
DVAR8: PhiY1  [ -89.95, -30 ]
DVAR9: PhiY2  [ -87.45, -25 ]
DVAR10: Phitau1  [ 1, 500 ]
DVAR11: Phitau2  [ 1, 500 ]
DVAR12: mY3ref  [ 1.105, 2.0 ]
DVAR13: PhiY3ref  [ -84.95, -20 ]
DVAR14: Eref  [ 0.055, 0.075 ]
```

In [3]:
# bounds of DVARS
lower_bound = [8.5, 5., 15., 1.005, 1.055, 1., 1., -89.95, -87.45, 1., 1., 1.105, -84.95, 0.055];
upper_bound = [12., 140., 160., 1.7, 1.85, 500., 500., -30., -25., 500., 500., 2.0, -20., 0.075];

# smapling to instantiate acquisition function
n_samples = 1000
initial_samples = Surrogates.sample(n_samples, lower_bound, upper_bound, SobolSample());

In [4]:
# scale
function minmaxscale(qty, lb, ub)
    2*(qty - lb)/(ub - lb) - 1
end

function inverseminmaxscale(qty_scaled, lb, ub)
    0.5*(ub - lb)*(qty_scaled + 1) + lb
end

inverseminmaxscale (generic function with 1 method)

In [5]:
# predictions and objective to minimize
function predict(x_vec)
    y_scaled = m([x for x in x_vec])'
end

function objective(x_vec)
    predict(x_vec)[5]
end

objective (generic function with 1 method)

In [6]:
# samples to instantiate acquisition function
xs = [Tuple(minmaxscale.(sample, lower_bound, upper_bound)) for sample in initial_samples];
ys = objective.(xs);

In [7]:
# surrogate for acquisition function
kriging_surrogate = Kriging(xs, ys, lower_bound, upper_bound, p=repeat([1.9], 14))

(::Kriging{Vector{NTuple{14, Float64}}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Vector{Float64}, Float64, Vector{Float64}, Float64, Matrix{Float64}}) (generic function with 2 methods)

In [10]:
# the optimization
optimum = surrogate_optimize(
    objective, LCBS(), lower_bound, upper_bound, kriging_surrogate, SobolSample(), maxiters=500
)

Out of sampling points


((-0.294921875, 0.646484375, 0.333984375, 0.8691406249999998, -0.2675781250000001, 0.236328125, -0.330078125, -0.8417968750000001, -0.9199218750000001, -0.533203125, -0.158203125, 0.43164062499999956, 0.337890625, 0.001953125), -1.0554024456579312)

In [13]:
inverseminmaxscale.(optimum[1], lower_bound, upper_bound)

14-element Vector{Float64}:
   9.73388671875
 116.1376953125
 111.7138671875
   1.6545263671875
   1.3461376953125
 309.4638671875
 168.1455078125
 -85.207861328125
 -84.949560546875
 117.4658203125
 211.0283203125
   1.7456591796874998
 -41.502001953125
   0.06501953125

In [14]:
predict(optimum[1])

1×6 adjoint(::Vector{Float64}) with eltype Float64:
 -1.02943  -0.210015  -0.362345  -0.933734  -1.0554  -0.132976